In [1]:
import pandas as pd
import re
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
from datetime import datetime

# Schlagworte zur NACE-Zuordnung
nace_schlagworte = {
    "A": ["Agriculture", "Food", "Palm Oil"],
    "B": ["Mining", "Oil", "Petroleum", "Raw Material"],
    "C13": ["Textiles", "Fashion", "Clothing"],
    "C20": ["Chemicals", "Pharmaceutical"],
    "C22": ["Plastics"],
    "C26": ["Electronics", "Technology"],
    "D": ["Utilities", "Waste", "Energy", "Power"],
    "G": ["Supply Chain", "Retail", "Trade"],
    "H": ["Transport"],
    "K": ["Financial", "Corporate Responsibility", "Investment"]
}

# Funktion zur Bewertung der Quelle nach 8 Kriterien
def score_sources(source_name, author_name, language, date_str, sector_relevant, has_sources, peer_review, independent):
    score = 0

    # 1. Quelle der Information (Autorität)
    good_sources = ["UN", "UBA"]
    if any(q.lower() in source_name.lower() for q in good_sources):
        score += 10
    else:
        score += 5

    # 2. Aktualität
    try:
        date = datetime.strptime(date_str, "%Y-%m-%d")
        tage_alt = (datetime.today() - date).days
        if tage_alt < 365:
            score += 10
        elif tage_alt < 1095:
            score += 8
        elif tage_alt < 2190:
            score += 6
        else:
            score += 3
    except:
        score += 5  # Fallback

    # 3. Autor-Kompetenz
    competent_authors = ["Dr.", "Prof.", "Expert", "Researcher"]
    if any(k.lower() in author_name.lower() for k in competent_authors):
        score += 10
    else:
        score += 6

    # 4. Transparenz
    score += 10 if has_sources else 5

    # 5. Überprüfung (Peer Review, staatlich etc.)
    score += 10 if peer_review else 4

    # 6. Sprache
    if language.lower() == "english":
        score += 10
    elif language.lower() == "deutsch" or language.lower() == "german":
        score += 7
    else:
        score += 5

    # 7. Branchenrelevanz
    score += 10 if sector_relevant else 5

    # 8. Unabhängigkeit
    score += 10 if independent else 4

    # Gesamt
    final_score = round(score / 8, 1)  # Durchschnitt von 8 Kriterien
    return final_score

# Analyse durchführen
def analyse_datei(pfad):
    try:
        df_input = pd.read_excel(pfad)
    except Exception as e:
        messagebox.showerror("Fehler beim Laden", str(e))
        return

    zuordnung = []

    for _, row in df_input.iterrows():
        title = str(row["Title"])
        author = str(row["Author"])
        source = str(row["Source"])
        language = str(row["Language"])
        date_str = str(row["Date"])

        sector_relevant = bool(row["Sector_Relevant"])
        has_sources = bool(row["Has_Sources"])
        peer_review = bool(row["Peer_Review"])
        independent = bool(row["Independent"])

        # NACE-Code zuordnen
        matched_codes = []
        for code, keywords in nace_schlagworte.items():
            if any(re.search(rf"\b{kw}\b", title, re.IGNORECASE) for kw in keywords):
                matched_codes.append(code)

        nace_codes = ", ".join(matched_codes) if matched_codes else "Unbestimmt"

        # Vertrauenswürdigkeit bewerten
        vertrauen = score_sources(
            source, author, language, date_str,
            sector_relevant, has_sources, peer_review, independent
        )

        # Ergebniszeile speichern
        zuordnung.append({
            "Title": title,
            "Author": author,
            "NACE_Zuordnung": nace_codes,
            "Vertrauens_Score": vertrauen
        })

    # DataFrame erstellen
    df_output = pd.DataFrame(zuordnung)
    df_output.to_excel("nace_quellenbewertung.xlsx", index=False)
    return df_output

def lade_datei():
    pfad = filedialog.askopenfilename(filetypes=[("Excel-Dateien", "*.xlsx")])
    if not pfad:
        return
    df = analyse_datei(pfad)
    if df is not None:
        for row in tree.get_children():
            tree.delete(row)
        for _, row in df.iterrows():
            tree.insert("", tk.END, values=list(row))
        messagebox.showinfo("Fertig", "Analyse abgeschlossen und gespeichert als 'nace_quellenbewertung.xlsx'")

# UI erstellen
root = tk.Tk()
root.title("NACE Quellenanalyse Tool")
root.geometry("1000x600")
root.configure(bg="#f0f4f8")

frame_top = tk.Frame(root, bg="#f0f4f8")
frame_top.pack(pady=20)

lbl_title = tk.Label(frame_top, text="NACE-Zuordnung und Vertrauensbewertung", font=("Arial", 16, "bold"), bg="#f0f4f8")
lbl_title.pack(pady=5)

btn_laden = tk.Button(frame_top, text="Excel-Datei laden & analysieren", command=lade_datei, font=("Arial", 12), bg="#4CAF50", fg="white", padx=10, pady=5)
btn_laden.pack()

frame_table = tk.Frame(root)
frame_table.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

columns = ["Title", "Author", "NACE_Zuordnung", "Vertrauens_Score"]
tree = ttk.Treeview(frame_table, columns=columns, show="headings", height=20)

style = ttk.Style()
style.configure("Treeview.Heading", font=("Arial", 11, "bold"))
style.configure("Treeview", font=("Arial", 10))

for col in columns:
    tree.heading(col, text=col)
    tree.column(col, anchor="w", width=230)
tree.pack(fill=tk.BOTH, expand=True)

root.mainloop()


2025-06-23 20:38:20.097 Python[38187:3357537] The class 'NSOpenPanel' overrides the method identifier.  This method is implemented by class 'NSWindow'
